<a href="https://colab.research.google.com/github/bignamu/python-AI/blob/main/dog_cat_classification2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/Pytorch/dataSet/cat_dog_data

/content/drive/MyDrive/Colab Notebooks/Pytorch/dataSet/cat_dog_data


In [ ]:
import glob
import os
import torchvision.transforms as transforms
import torchvision
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import cv2
import torch
from torchvision import datasets, models, transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader

In [ ]:
# torch version gpu 사용 유무 체크 
print("torch version >> " , torch.__version__)
print("GPU 사용 유무 >> " , torch.cuda.is_available())

torch version >>  1.9.0+cu102
GPU 사용 유무 >>  True


In [ ]:
# device 설정 GPU 사용 가능 여부에 따라 device 정보 저장 
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = torch.device(device)

In [ ]:
# data check function 
def dataset_check(output_dir) : 
  try :

    # output_dir /content/drive/MyDrive/AI 스쿨자료/cat_dog_data/train/cat/*.jpg
    # train data check
    all_train_cat = glob.glob(os.path.join(output_dir, "train", "cat", "*.jpg"))
    all_train_dog = glob.glob(os.path.join(output_dir, "train", "dog", "*.jpg"))


    # val data check 
    all_val_cat = glob.glob(os.path.join(output_dir, "val", "cat", "*.jpg"))
    all_val_dog = glob.glob(os.path.join(output_dir, "val", "dog", "*.jpg"))

    # test data check 
    all_test_cat = glob.glob(os.path.join(output_dir, "test", "cat", "*.jpg"))
    all_test_dog = glob.glob(os.path.join(output_dir, "test", "dog", "*.jpg"))

    # print show 
    print("train cat image number >> " , len(all_train_cat))
    print("train dog image number >> " , len(all_train_dog))

    print("val cat image number >> ", len(all_val_cat))
    print("val dog image number >> ", len(all_val_dog))

    print("test cat image number >> ", len(all_test_cat))
    print("test dog image number >> ", len(all_test_dog))

  except Exception as e :
    print("check error 코드 구현을 완료 했는지 검토 하시오 !! ")
    print(e)
  # data location 
data_dir = "/content/drive/MyDrive/Colab Notebooks/Pytorch/dataSet/cat_dog_data"

# data check 
dataset_check(data_dir)


train cat image number >>  1000
train dog image number >>  1000
val cat image number >>  500
val dog image number >>  500
test cat image number >>  1000
test dog image number >>  1000


In [ ]:
# data location 
data_dir = "/content/drive/MyDrive/Colab Notebooks/Pytorch/dataSet/cat_dog_data"

# data check 
dataset_check(data_dir)

train cat image number >>  1000
train dog image number >>  1000
val cat image number >>  500
val dog image number >>  500
test cat image number >>  1000
test dog image number >>  1000


In [ ]:
# hyper parameter setting 
batch_size = 32 
num_epochs = 40
lerning_rate = 0.01

In [ ]:
class catdogDataset(Dataset) : 
  def __init__(self,data_dir, mode, transform=None) :
    # 데이터 정의 
    # data_dir/train/cat/images
    self.all_data = sorted(glob.glob(os.path.join(data_dir, mode, "*" , "*")))
    self.transform = transform

  def __getitem__(self, item) :
    # 데이터 이미지 읽기 
    data_path = self.all_data[item]
    
    # 이미지 읽기 
    img = Image.open(data_path)
    img = img.convert("RGB")

    # 라벨 읽기
    label_name = os.path.basename(data_path)
    label_str = str(label_name)

    if label_str.startswith("cat") == True : 
      label = 0 
    else :
      label = 1
    
    # transform is not None 
    if self.transform is not None : 
      img = self.transform(img)

    return img, label

  def __len__(self) :
    length = len(self.all_data)
    return length

In [ ]:
# data transforms 
# image resiz -> 224 244 
data_transforms = {
    'train' : transforms.Compose([
        transforms.RandomRotation(5),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.Resize([224,224]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])                     
    ]),
    'val' : transforms.Compose([
        transforms.Resize([224,224]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test' : transforms.Compose([
        transforms.Resize([224,224]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])                         
    ])
}

In [ ]:
# 데이터 정의 및 데이터 loader 코드 구현 
# data set (train, val, test)
train_data_set = catdogDataset(data_dir, mode='train', transform=data_transforms['train'])
val_data_set = catdogDataset(data_dir, mode='val', transform=data_transforms['val'])
test_data_set = catdogDataset(data_dir, mode="test", transform=data_transforms['test'])

# data loader 
train_loader = DataLoader(train_data_set, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_data_set, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader = DataLoader(test_data_set, batch_size=batch_size, shuffle=False, drop_last=True)


In [ ]:
# 간단한 모델 구성
class SimpleCNN(nn.Module):
  def __init__(self):
    super(SimpleCNN,self).__init__()

    # self vision
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(2)

    )
    # fc1 fc2
    self.fc1 = nn.Linear(256 * 5 * 5, 512)
    self.fc2 = nn.Linear(512,2)

  def forward(self,x):
      x = self.conv(x)
      x = x.view(x.shape[0],-1) # 원핫인코딩?
      x = F.relu(self.fc1(x))
      x = self.fc2(x)

      return x

In [ ]:
# net = SimpleCNN().to(device)
# print(net)

net = models.resnet18(pretrained=True).to(device)
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs,2).to(device)


# print(net)

In [ ]:
#loss function, optimizer scheduler
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(net.parameters(),lr=lerning_rate,momentum=0.9)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=20,gamma=0.1)


In [ ]:
def train(num_epochs,model,data_loader,criterion,optimizer,save_dir,val_every,device):
  
  print("Start train .... !!!")
  best_loss = 9999
  for epoch in range(num_epochs):
    for i, (imgs,labels) in enumerate(data_loader):
      images,labels = imgs.to(device) , labels.to(device)

      outputs = net(images)

      loss = criterion(outputs,labels)
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      _, argmax = torch.max(outputs,1)
      accuracy = (labels == argmax).float().mean()

      if (i + 1) % 3 == 0:
        print("Epoch[{}/{}], step[{},{}], loss [{:.4f}],Accuracy: [{:.2f}%]"
        .format(epoch + 1,num_epochs,i+1,len(train_loader),loss.item(),accuracy.item()* 100))

    if (epoch + 1) % val_every == 0:
      avrg_loss = validation(epoch + 1, model,val_loader,criterion,device)      

      if avrg_loss < best_loss:
        print("Best performance at epoch : {}".format(epoch+1))
        print("Save model in ",save_dir)
        best_loss = avrg_loss
        save_model(model,save_dir)



In [ ]:
# def validation(epoch,model,data_loader,criterion,device):
#   print("Start val")
#   model.eval()

#   with torch.no_grad():
#     total = 0
#     correct = 0
#     total_loss = 0
#     cnt = 0

#     for i, (imgs,labels) in enumerate(data_loader):
#       images,labels=imgs.to(device),labels.to(device)
#       output = net(images)
#       loss = criterion(output,labels)
#       total += imgs.size(0)
#       _, argmax = torch.max(output,1)
#       accuracy = (labels == argmax).float().mean()
      
#       correct += (labels == argmax).sum().item()
#       total_loss += loss
#       cnt += 1

#     avrg_loss = total_loss / cnt
#     print("Validation #{} Accuracy {:.2f} % Average Loss : {.4f}".format(epoch,correct / total * 100,avrg_loss))
#   model.train()

#   return avrg_loss

def validation(epoch, model, data_loader, criterion, device) :
  print("Start val")
  model.eval()

  with torch.no_grad():
    total = 0
    correct = 0
    total_loss = 0
    cnt = 0

    for i, (imgs, labels) in enumerate(data_loader) :
      images , labels = imgs.to(device) , labels.to(device)
      output = net(images)
      loss = criterion(output, labels)

      total += imgs.size(0)
      _, argmax = torch.max(output, 1)
      accuracy = (labels == argmax).float().mean()
      
      correct += (labels == argmax).sum().item()
      total_loss += loss
      cnt +=1

    avrg_loss = total_loss / cnt 
    print("Validation #{} Accuracy {:.2f}% Average Loss : {:.4f}".format(epoch, correct / total * 100, avrg_loss))
  
  model.train()

  return avrg_loss



In [ ]:
# save model
def save_model(model,save_dir,file_name = "best_model.pt"):
  os.makedirs(save_dir,exist_ok=True)
  output_path = os.path.join(save_dir,file_name)
  torch.save(net.state_dict(),output_path)


In [ ]:
# test
def test(model,data_loader,device):
  print("start test")

  correct = 0
  total = 0
  with torch.no_grad():
    for i,(imgs,labels) in enumerate(data_loader):
      imgs,labels=imgs.to(device) , labels.to(device)

      outputs = net(imgs)
      _, argmax = torch.max(outputs,1)
      total += imgs.size(0)
      correct += (labels == argmax).sum().item()

    print("Test Accuracy for {} images : {:.2f}%".format(total,correct / total * 100))



In [ ]:
model = net
val_every = 1
save_dir = "./weights/"

train(num_epochs,model,train_loader,criterion,optimizer,save_dir,val_every,device)

Start train .... !!!


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch[1/40], step[3,62], loss [0.5539],Accuracy: [78.12%]
Epoch[1/40], step[6,62], loss [0.3049],Accuracy: [84.38%]
Epoch[1/40], step[9,62], loss [0.3711],Accuracy: [84.38%]
Epoch[1/40], step[12,62], loss [0.2414],Accuracy: [87.50%]
Epoch[1/40], step[15,62], loss [0.0859],Accuracy: [96.88%]
Epoch[1/40], step[18,62], loss [0.1895],Accuracy: [93.75%]
Epoch[1/40], step[21,62], loss [0.1042],Accuracy: [93.75%]
Epoch[1/40], step[24,62], loss [0.3343],Accuracy: [84.38%]
Epoch[1/40], step[27,62], loss [0.1621],Accuracy: [93.75%]
Epoch[1/40], step[30,62], loss [0.8249],Accuracy: [78.12%]
Epoch[1/40], step[33,62], loss [0.6592],Accuracy: [78.12%]
Epoch[1/40], step[36,62], loss [0.2384],Accuracy: [93.75%]
Epoch[1/40], step[39,62], loss [0.1223],Accuracy: [96.88%]
Epoch[1/40], step[42,62], loss [0.4047],Accuracy: [87.50%]
Epoch[1/40], step[45,62], loss [0.3831],Accuracy: [87.50%]
Epoch[1/40], step[48,62], loss [0.6642],Accuracy: [90.62%]
Epoch[1/40], step[51,62], loss [0.2553],Accuracy: [87.50%]


In [ ]:
pwd

'/content/drive/MyDrive/Colab Notebooks/Pytorch/dataSet/cat_dog_data'